<a href="https://colab.research.google.com/github/Azimoj/WebScraping/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtenir les derniers articles du Monde (**)

Obtenir le titre et le chapeau (description) des derniers articles sur https://www.lemonde.fr/actualite-en-continu/ (*)

Enregistrer les images correspondantes (***, @)

In [4]:
import requests
from bs4 import BeautifulSoup

 Obtenir le titre et les chapeaux

La première étape est de demander la page au serveur en utilisant la fonction requests.get.

In [5]:
page= requests.get("https://www.lemonde.fr/actualite-en-continu/")

In [6]:
page.status_code

200

Noter le code de réponse : 200 = Success.

On peut afficher le contenu HTML de la page avec l'attribut content.

In [ ]:
page.content

In [ ]:
soup = BeautifulSoup(page.content)

print(soup.prettify()) # pour un meilleur affichage

On peut ensuite extraire le texte depuis les tags qui nous intéressent.

Pour connaître le noms des tags pertinents, on utilise l'inspecteur du navigateur. On va se concentrer sur la section id="river", qui contient les articles :

In [ ]:
content = soup.find('section', {'id':'river'})
content

In [12]:
# Syntaxe alternative : la méthode select

content2 = soup.select("html body section#river .teaser")

# Cette méthode utilise la syntaxe CSS pour rechercher un tag.
# Elle est parfois plus rapide à écrire. Les tags successifs sont
# separés par des espaces, les # désignent une class et les . un id.
# On peut obtenir le chemin CSS avec clic droit dans l'inspecteur.

On isole les titres (h3) et les chapeaux (p, class='teaser__desc') des articles :

In [17]:
titres = content.find_all('h3')
titres

[<h3 class="teaser__title">Bronchiolite : la distribution du Beyfortus interrompue dans les pharmacies de ville</h3>,
 <h3 class="teaser__title">Ilia Iachine, opposant russe : « La véritable Russie ne combat pas les Ukrainiens sur le front. La véritable Russie est en prison »</h3>,
 <h3 class="teaser__title">Affaire Benalla : peine confirmée en appel pour Alexandre Benalla après les « violences du 1ᵉʳ-Mai »</h3>,
 <h3 class="teaser__title">Paris 2024 : les Russes ne sont pas exclus des Jeux paralympiques</h3>,
 <h3 class="teaser__title">L’inflation en France reste à un niveau élevé, la consommation des ménages en baisse, rapporte l’Insee</h3>,
 <h3 class="teaser__title">Epic Games, société créatrice du jeu vidéo « Fortnite », annonce le licenciement de plus de 800 personnes</h3>,
 <h3 class="teaser__title">Les Etats-Unis vont-ils connaître un nouveau « shutdown » ? Ce qu’il faut savoir sur cette bataille budgétaire et ses conséquences</h3>,
 <h3 class="teaser__title">Plus de 50 morts e

In [16]:
chapeaux = content.find_all('p', {'class': 'teaser__desc'})
chapeaux

[<p class="teaser__desc">L’engouement pour le nouveau traitement est tel que les commandes passées par les officines tardent à arriver. La direction générale de la santé a recommandé, vendredi 29 septembre, aux médecins de ville de suspendre les prescriptions du dosage pour les bébés de plus de 5 kilos.</p>,
 <p class="teaser__desc">L’homme politique membre de l’opposition, condamné à huit ans et demi de colonie pénitentiaire pour avoir dénoncé l’agression russe en Ukraine, a envoyé un texte à l’occasion d’un forum organisé par l’association Russie-Libertés, à Paris, les 29 et 30 septembre. « Le Monde » en publie de larges extraits.</p>,
 <p class="teaser__desc">En première instance, l’ancien collaborateur de l’Elysée avait été condamné à trois ans de prison, dont un an ferme sous bracelet électronique.</p>,
 <p class="teaser__desc">Le comité international paralympique a voté contre l’exclusion vendredi du comité russe. Une décision qui marque un infléchissement de la fermeté dont a fa

In [14]:
titres[0].get_text()

NameError: ignored

In [ ]:
# Nettoyage sous forme de list comprehension (mieux)
titres = [ unidecode(t.get_text()) for t in titres ]
titres